In [1]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np

parent_dir = os.path.dirname(os.getcwd())
print(parent_dir)

/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24


In [2]:
loaded_model = tf.keras.models.load_model(
os.path.join(parent_dir, 'Data Embedding & Model Development/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Fold 2.keras'))

ValueError: File not found: filepath=/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Data Embedding & Model Development/Classification Models/Complete Set 1 Models/ROBOMechDB Complete Set 1 Model Fold 2.keras. Please ensure the file is an accessible `.keras` zip file.

In [4]:
#loading triple embeddings from robokops kg
df = pd.read_csv(os.path.join(parent_dir,'Data Embedding & Model Development/Data Mined Embedded Dataset/ROBOKOP Virtual Screening Set Embedded Dataset.csv')
)
df = df.drop('Unnamed: 0', axis = 1)
df = df.drop_duplicates(subset = ['0','2','4'])


In [3]:
print(df)

NameError: name 'df' is not defined

In [4]:
data = df.iloc[:, 6:-1]

print(data[:1])


NameError: name 'df' is not defined

In [5]:
predictions = loaded_model.predict(data)
data_rounded = np.round(predictions)
descending_confidence_values = np.sort(predictions.flatten())[::-1]
ascending_indices = np.argsort(predictions.flatten())
descending_indices = np.flip(ascending_indices)

NameError: name 'loaded_model' is not defined

In [6]:
print(ascending_indices[:5])

NameError: name 'ascending_indices' is not defined

In [7]:
print(predictions[1920])

NameError: name 'predictions' is not defined

In [8]:
print(df.iloc[1920])

NameError: name 'df' is not defined

In [9]:
## finding the indices of only the 1's (actives)

true_positive_indices = []
true_positive_count = 0

false_negative_indices = []
false_negative_count = 0

for i in range (0,len(data_rounded)):
    if data_rounded[i] == 1:
        true_positive_indices.append(i)
        true_positive_count += 1
    else:
        false_negative_indices.append(i)
        false_negative_count +=1

print(true_positive_indices[:20])
print(true_positive_count)
print(false_negative_indices[:20])
print(false_negative_count)

NameError: name 'data_rounded' is not defined

In [10]:
print(df.iloc[1,:6])

NameError: name 'df' is not defined

In [11]:
prediction_array = []
for i in descending_indices:
    triple_name = df.iloc[i,:6].tolist()
    conf_value = float(predictions[i])
    row = [triple_name,conf_value]
    flat_row = row[0]+[row[1]]
    prediction_array.append(flat_row)



NameError: name 'descending_indices' is not defined

In [12]:
print(prediction_array[:2])

[]


In [15]:
columns_to_use = ['drug_name','drug_id','disease_name','disease_id','protein_name','protein_id','model_confidence_value']
df = pd.DataFrame(data = prediction_array,columns = columns_to_use)
df = df.drop_duplicates(subset= ['drug_name','disease_name','protein_name'])
df.to_csv(os.path.join(os.getcwd(),'Screening Results/Complete Set 1 Top Model Predictions of Top ROBOKOP Data Mined Triples.csv'))

In [26]:
print(df)

             drug_name       drug_id               disease_name  \
0            promazine    CHEBI:8459  major depressive disorder   
2          hydroxyzine    CHEBI:5818  major depressive disorder   
4      brompheniramine    CHEBI:3183  major depressive disorder   
6            bromazine   CHEBI:59177  major depressive disorder   
7             loxapine   CHEBI:50841        depressive disorder   
...                ...           ...                        ...   
89580     dexibuprofen   CHEBI:43415                     ptosis   
89581     pimavanserin  CHEBI:133017        gingival overgrowth   
89582     dexibuprofen   CHEBI:43415                  exotropia   
89583      risperidone    CHEBI:8871               cornea plana   
89584     pimavanserin  CHEBI:133017               cornea plana   

          disease_id                   protein_name protein_id  \
0      MONDO:0002009          histamine receptor h1     P35367   
2      MONDO:0002009          histamine receptor h1     P35367 